# Good Deal / Bad Deal Indicator

**In This Notebook**: Create a simple price estimate using Linear Regression.

**In Tableau**: Visualize price estimates vs actual prices to determine what is a good deal vs bad deal.

## 1. Read in Chicago Listings

In [1]:
# import libraries
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
# read in Chicago listings
data_chicago = pd.read_csv('../Exercises/redfin_listings_chicago.csv')
data_chicago.head()

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
0,MLS Listing,NaN,Condo/Co-op,1335 S Prairie Ave #1105,Chicago,IL,60605,799000,3.0,2.5,...,Active,NaN,NaN,http://www.redfin.com/IL/Chicago/1335-S-Prairi...,MRED,11268174,N,Y,41.864674,-87.620765
1,MLS Listing,NaN,Single Family Residential,4944 W Roscoe St,Chicago,IL,60641,424900,4.0,3.5,...,Active,NaN,NaN,http://www.redfin.com/IL/Chicago/4944-W-Roscoe...,MRED,11268165,N,Y,41.942626,-87.751249
2,MLS Listing,NaN,Single Family Residential,8111 S Scottsdale Ave,Chicago,IL,60652,265000,3.0,1.0,...,Active,NaN,NaN,http://www.redfin.com/IL/Chicago/8111-S-Scotts...,MRED,11268170,N,Y,41.745964,-87.737172
3,MLS Listing,NaN,Condo/Co-op,4335 N Mozart St #1,Chicago,IL,60618,325000,3.0,2.0,...,Active,November-14-2021 11:00 AM,November-14-2021 12:30 PM,http://www.redfin.com/IL/Chicago/4335-N-Mozart...,MRED,11266874,N,Y,41.960457,-87.699344
4,MLS Listing,NaN,Single Family Residential,5736 S Loomis Blvd,Chicago,IL,60636,159900,5.0,2.0,...,Active,NaN,NaN,http://www.redfin.com/IL/Chicago/5736-S-Loomis...,MRED,11268151,N,Y,41.789370,-87.659866


In [3]:
# only look at numeric values to create a simple model
data_subset = data_chicago[["PRICE", "BEDS", "BATHS", "SQUARE FEET", "YEAR BUILT"]]
data_subset = data_subset.dropna()
data_subset

,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT
0,799000,3.0,2.5,2076.0,2003.0
1,424900,4.0,3.5,2921.0,1924.0
2,265000,3.0,1.0,998.0,1956.0
3,325000,3.0,2.0,1875.0,2002.0
4,159900,5.0,2.0,1953.0,1906.0
...,...,...,...,...,...
337,239000,2.0,2.0,1100.0,1969.0
338,649000,3.0,2.0,1700.0,2004.0
339,265000,1.0,1.0,770.0,1924.0
341,245000,3.0,2.0,1200.0,1926.0


## 2. Fit a Model to Predict Home Price

In [4]:
# set the dependent variable / output as price
y = data_subset.PRICE
y

0      799000
1      424900
2      265000
3      325000
4      159900
        ...  
337    239000
338    649000
339    265000
341    245000
342    190000
Name: PRICE, Length: 199, dtype: int64

In [5]:
# set the independent variables / inputs as the numeric fields
X = data_subset[["BEDS", "BATHS", "SQUARE FEET", "YEAR BUILT"]]
X

,BEDS,BATHS,SQUARE FEET,YEAR BUILT
0,3.0,2.5,2076.0,2003.0
1,4.0,3.5,2921.0,1924.0
2,3.0,1.0,998.0,1956.0
3,3.0,2.0,1875.0,2002.0
4,5.0,2.0,1953.0,1906.0
...,...,...,...,...
337,2.0,2.0,1100.0,1969.0
338,3.0,2.0,1700.0,2004.0
339,1.0,1.0,770.0,1924.0
341,3.0,2.0,1200.0,1926.0


In [6]:
# train/test split before fitting model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1118)

In [7]:
# fit a linear regression model
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

## 3. Look at the Metrics

In [8]:
# view R-squared
lr.score(X_test, y_test)

0.6502841724507231

In [9]:
# view RMSE (root mean squared error)
actual = y_test
predicted = lr.predict(X_test)
mean_squared_error(actual, predicted)**.5

205458.48700571858

## 4. View Price Estimates

In [10]:
# price estimates for each home based on model
price_estimates = lr.predict(X)
data_subset["PRICE_ESTIMATE"] = price_estimates.tolist()
data_subset

,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,PRICE_ESTIMATE
0,799000,3.0,2.5,2076.0,2003.0,7.262737e+05
1,424900,4.0,3.5,2921.0,1924.0,1.007532e+06
2,265000,3.0,1.0,998.0,1956.0,1.394202e+04
3,325000,3.0,2.0,1875.0,2002.0,5.557068e+05
4,159900,5.0,2.0,1953.0,1906.0,2.537927e+05
...,...,...,...,...,...,...
337,239000,2.0,2.0,1100.0,1969.0,3.813045e+05
338,649000,3.0,2.0,1700.0,2004.0,5.023633e+05
339,265000,1.0,1.0,770.0,1924.0,1.235892e+05
341,245000,3.0,2.0,1200.0,1926.0,2.553778e+05


In [11]:
## merge back with original data set
price_estimates = pd.merge(data_chicago, data_subset, left_index=True, right_index=True, suffixes=(None, '_Duplicate'))
price_estimates

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,PRICE_Duplicate,BEDS_Duplicate,BATHS_Duplicate,SQUARE FEET_Duplicate,YEAR BUILT_Duplicate,PRICE_ESTIMATE
0,MLS Listing,NaN,Condo/Co-op,1335 S Prairie Ave #1105,Chicago,IL,60605,799000,3.0,2.5,...,N,Y,41.864674,-87.620765,799000,3.0,2.5,2076.0,2003.0,7.262737e+05
1,MLS Listing,NaN,Single Family Residential,4944 W Roscoe St,Chicago,IL,60641,424900,4.0,3.5,...,N,Y,41.942626,-87.751249,424900,4.0,3.5,2921.0,1924.0,1.007532e+06
2,MLS Listing,NaN,Single Family Residential,8111 S Scottsdale Ave,Chicago,IL,60652,265000,3.0,1.0,...,N,Y,41.745964,-87.737172,265000,3.0,1.0,998.0,1956.0,1.394202e+04
3,MLS Listing,NaN,Condo/Co-op,4335 N Mozart St #1,Chicago,IL,60618,325000,3.0,2.0,...,N,Y,41.960457,-87.699344,325000,3.0,2.0,1875.0,2002.0,5.557068e+05
4,MLS Listing,NaN,Single Family Residential,5736 S Loomis Blvd,Chicago,IL,60636,159900,5.0,2.0,...,N,Y,41.789370,-87.659866,159900,5.0,2.0,1953.0,1906.0,2.537927e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,MLS Listing,NaN,Condo/Co-op,3930 N Pine Grove Ave #1515,Chicago,IL,60613,239000,2.0,2.0,...,N,Y,41.954237,-87.648457,239000,2.0,2.0,1100.0,1969.0,3.813045e+05
338,MLS Listing,NaN,Condo/Co-op,1359 N Mohawk St #3,Chicago,IL,60610,649000,3.0,2.0,...,N,Y,41.907243,-87.641827,649000,3.0,2.0,1700.0,2004.0,5.023633e+05
339,MLS Listing,NaN,Condo/Co-op,310 S Michigan Ave #910,Chicago,IL,60604,265000,1.0,1.0,...,N,Y,41.877825,-87.624602,265000,1.0,1.0,770.0,1924.0,1.235892e+05
341,MLS Listing,NaN,Single Family Residential,70 E 101st St,Chicago,IL,60628,245000,3.0,2.0,...,N,Y,41.711045,-87.620971,245000,3.0,2.0,1200.0,1926.0,2.553778e+05


## 5. Export Price Estimates

In [12]:
# export data to read into Tableau
price_estimates.to_csv('chicago_price_estimates.csv')